In [2]:
# Import libraries and packages for the project 
# import selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup#pulling data out of HTML files
from time import sleep#slow down the execution to pass the security check
import csv
import pandas as pd

In [3]:
df=pd.read_csv('data/CUNY_students.csv')
df.head()

UnicodeDecodeError: 'utf-8' codec can't decode byte 0x90 in position 22: invalid start byte

good Youtube videos:
1. [link]https://www.youtube.com/watch?v=zkfLAY2OrtI
2. [link]https://www.linkedin.com/learning/python-automation-and-testing/what-is-the-html-dom-structure?autoplay=true&resume=false

In [2]:
# Task 1: Login to Linkedin

credential=open('login_credential.txt')
line=credential.readlines()
username=line[0]
password=line[1]

# Task 1.1: Open Chrome and Access Linkedin login site
driver =webdriver.Chrome()
url = 'https://www.linkedin.com/login'
driver.get(url)
sleep(2)

# Task 1.2: Key in login credentials
email_field = driver.find_element('name','session_key')
email_field.send_keys(username)
sleep(2)

password_field = driver.find_element('id','password')
password_field.send_keys(password)
sleep(2)

# Task 1.2: Click the Login button
login_field = driver.find_element('xpath',"/html/body/div/main/div[2]/div[1]/form/div[3]/button")
login_field.click()
sleep(3)

In [3]:
# Task 2: Search for the profile we want to crawl

# Task 2.1: Locate the search bar element
search_field = driver.find_element('xpath',"/html/body/div[5]/header/div/div/div/div[1]/input")
search_field.click()
# Task 2.2: Input the search query to the search bar
search_query = input("what do you want to search:")#using the list from Dr C
search_field.send_keys(search_query)

# Task 2.3: Search
search_field.send_keys(Keys.RETURN)



what do you want to search:Jiale Chen


In [5]:
# Task 3: Scrape the URLs of the profiles

# Task 3.1: Write a function to extract the URLs of one page
def GetURL():
    page_source = BeautifulSoup(driver.page_source)
    profiles = page_source.find_all('div',class_="entity-result__item") 


    all_profile_url=[]
    for profile in profiles:
        profile_url=profile.find('a',class_="app-aware-link")['href']
        if profile_url not in all_profile_url:
            all_profile_url.append(profile_url)
    return all_profile_url

# print(GetURL())

   


# Task 3.2: Navigate through the next pages, and extract the profile URLs of each page

def GetURL_pages():
    input_page = 1
    URLs_all_page = []
    for page in range(input_page):
        URLs_one_page = GetURL()
        driver.execute_script('window.scrollTo(0,document.body.scrollHeight)')#we need to scroll down to Next page button
        sleep(1)
        next_button = driver.find_element('xpath',"//button[@aria-label='Next'][@type='button']").click()
        URLs_all_page = URLs_all_page + URLs_one_page
        
        sleep(1)
    return URLs_all_page

URLs_all_page=GetURL_pages()
print(URLs_all_page)

['https://www.linkedin.com/in/jialechen119?miniProfileUrn=urn%3Ali%3Afs_miniProfile%3AACoAAC3-p7sBioNafow2j51hvc6himxBx3Hcxrs', 'https://www.linkedin.com/in/jiale-chen-7765b5207?miniProfileUrn=urn%3Ali%3Afs_miniProfile%3AACoAADSXjQoB3SEWcoHvZLC0oeQTYPHQU12qXGc', 'https://www.linkedin.com/in/jialec?miniProfileUrn=urn%3Ali%3Afs_miniProfile%3AACoAADqwa-ABoWgUbXXY_p9U8iIfHvzToJhW1x4', 'https://www.linkedin.com/in/jiale-chen?miniProfileUrn=urn%3Ali%3Afs_miniProfile%3AACoAACqVTTEB8_z-LXnTvSzmNfofWkWa7NVsU5Y', 'https://www.linkedin.com/in/jiale-chen-a315b8103?miniProfileUrn=urn%3Ali%3Afs_miniProfile%3AACoAABpAYsUB4AjDbVyqTMR7Qj8OC9CCMRWOkfw', 'https://www.linkedin.com/in/jiale-chen-29b639a7?miniProfileUrn=urn%3Ali%3Afs_miniProfile%3AACoAABayaVcBSxyZx9J7irS1PDUFjWN247KQ_FU', 'https://www.linkedin.com/in/jia-le-chen-bb11ab15b?miniProfileUrn=urn%3Ali%3Afs_miniProfile%3AACoAACZLNvUB2F-fwwdMGxH6NIgSW4LuSdmjKU8', 'https://www.linkedin.com/in/jiale-jalen-chen-8a025521a?miniProfileUrn=urn%3Ali%3Afs_m

In [78]:
URLs_all_page[0]

'https://www.linkedin.com/in/jialechen119?miniProfileUrn=urn%3Ali%3Afs_miniProfile%3AACoAAC3-p7sBioNafow2j51hvc6himxBx3Hcxrs'

In [6]:
# Task 4: Scrape the data of 1 Linkedin profile, and write the data to a .CSV file
with open('output.csv', 'w',  newline = '') as file_output:
    headers = ['Name', 'Job Title', 'Location', 'URL']
    writer = csv.DictWriter(file_output, delimiter=',', lineterminator='\n',fieldnames=headers)
    writer.writeheader()
    
    for linkedin_URL in URLs_all_page:
        driver.get(linkedin_URL)
        print(linkedin_URL)
        sleep(1)
        page_source = BeautifulSoup(driver.page_source, "html.parser")
        info_div = page_source.find('div',{'class':'ph5 pb5'})

        name = info_div.find('h1', class_='text-heading-xlarge inline t-24 v-align-middle break-words').get_text().strip() #Remove unnecessary characters 
        #print('--- Profile name is: ', name)
        location = info_div.find('span', class_='text-body-small inline t-black--light break-words').get_text().strip() #Remove unnecessary characters 
        #print('--- Profile location is: ', location)
        title = info_div.find('div', class_='text-body-medium break-words').get_text().strip()
        #print('--- Profile title is: ', title)
        writer.writerow({headers[0]:name, headers[1]:location, headers[2]:title, headers[3]:linkedin_URL})
        print('\n')
    
        



https://www.linkedin.com/in/jialechen119?miniProfileUrn=urn%3Ali%3Afs_miniProfile%3AACoAAC3-p7sBioNafow2j51hvc6himxBx3Hcxrs


https://www.linkedin.com/in/jiale-chen-7765b5207?miniProfileUrn=urn%3Ali%3Afs_miniProfile%3AACoAADSXjQoB3SEWcoHvZLC0oeQTYPHQU12qXGc


https://www.linkedin.com/in/jialec?miniProfileUrn=urn%3Ali%3Afs_miniProfile%3AACoAADqwa-ABoWgUbXXY_p9U8iIfHvzToJhW1x4


https://www.linkedin.com/in/jiale-chen?miniProfileUrn=urn%3Ali%3Afs_miniProfile%3AACoAACqVTTEB8_z-LXnTvSzmNfofWkWa7NVsU5Y


https://www.linkedin.com/in/jiale-chen-a315b8103?miniProfileUrn=urn%3Ali%3Afs_miniProfile%3AACoAABpAYsUB4AjDbVyqTMR7Qj8OC9CCMRWOkfw


https://www.linkedin.com/in/jiale-chen-29b639a7?miniProfileUrn=urn%3Ali%3Afs_miniProfile%3AACoAABayaVcBSxyZx9J7irS1PDUFjWN247KQ_FU


https://www.linkedin.com/in/jia-le-chen-bb11ab15b?miniProfileUrn=urn%3Ali%3Afs_miniProfile%3AACoAACZLNvUB2F-fwwdMGxH6NIgSW4LuSdmjKU8


https://www.linkedin.com/in/jiale-jalen-chen-8a025521a?miniProfileUrn=urn%3Ali%3Afs_miniProfil